# Parse TypedDict Notation Using Abstract Syntax Tree

In [1]:
import ast

In [2]:
# type_def.pyi

import typing as T
from typing import (
    TypedDict,
    Optional,
)

a: int = 1
b: str = "Alice"

class User(TypedDict):
    id: int
    name: str

In [3]:
source = """
# type_def.pyi

import typing as T
from typing import (
    TypedDict,
    Optional,
)

a: int = 1
b: str = "Alice"

class User(TypedDict):
    id: int
    name: str
""".strip()

module = ast.parse(source)

In [4]:
for i, node in enumerate(module.body, start=1):
    print(f"========== {i} ==========")
    text = ast.get_source_segment(source, node)
    print(text)
    print(f"{type(node) = }")

========== 1 ==========
import typing as T
type(node) = <class 'ast.Import'>
========== 2 ==========
from typing import (
    TypedDict,
    Optional,
)
type(node) = <class 'ast.ImportFrom'>
========== 3 ==========
a: int = 1
type(node) = <class 'ast.AnnAssign'>
========== 4 ==========
b: str = "Alice"
type(node) = <class 'ast.AnnAssign'>
========== 5 ==========
class User(TypedDict):
    id: int
    name: str
type(node) = <class 'ast.ClassDef'>


## 最简单的赋值

``ast.Assign`` 是赋值. 类似于 ``a = 1`` (不带类型注解)

- ``targets``: 等式左边的对象, 因为 Python 支持多个变量同时赋值, 所以它是一个列表. 通常里面只有一个元素.
- ``targets[0]``: 这里是 ``ast.Name``, 也就是变量名.
- ``value``: 等式右边的值. 它有 value 和 kind 两个属性. 其中 value 的部分非常重要.

In [5]:
# type_def.pyi

a = 1

In [6]:
source = """
# type_def.pyi

a = 1
""".strip()

module = ast.parse(source)
assign = module.body[0]
print(f"{assign = }")
print(f"{assign.targets = }")
print(f"{assign.targets[0] = }")
print(f"{assign.targets[0].id = }")
print(f"{assign.targets[0].ctx = }")
print(f"{assign.value = }")
print(f"{assign.value.value = }")
print(f"{assign.value.kind = }")
print(f"{assign.type_comment = }")

assign = <ast.Assign object at 0x103767d60>
assign.targets = [<ast.Name object at 0x103767d00>]
assign.targets[0] = <ast.Name object at 0x103767d00>
assign.targets[0].id = 'a'
assign.targets[0].ctx = <ast.Store object at 0x100a47fd0>
assign.value = <ast.Constant object at 0x103767a60>
assign.value.value = 1
assign.value.kind = None
assign.type_comment = None


## 带类型标注的赋值

``ast.AnnAssign`` 是赋值. 类似于 ``a = 1`` (带类型注解)

跟普通的 Assign 赋值的不同之处有:

- 只有一个 target. 这很好理解, 因为类型标注只标注一个.
- 多了一个 ``annotation`` 的属性, 储存的是类型注解信息. 这个属性的值取决于你是如何标注的.

In [7]:
# type_def.pyi

a: int = 1

In [8]:
source = """
# type_def.pyi

a: int = 1
""".strip()

module = ast.parse(source)
assign = module.body[0]
print(f"{assign = }")
print(f"{assign.target = }")
print(f"{assign.target.id = }")
print(f"{assign.target.ctx = }")
print(f"{assign.annotation = }")
print(f"{assign.annotation.id = }")
print(f"{assign.annotation.ctx = }")
print(f"{assign.value = }")
print(f"{assign.value.value = }")
print(f"{assign.value.kind = }")

assign = <ast.AnnAssign object at 0x103767f70>
assign.target = <ast.Name object at 0x103767fd0>
assign.target.id = 'a'
assign.target.ctx = <ast.Store object at 0x100a47fd0>
assign.annotation = <ast.Name object at 0x103765d20>
assign.annotation.id = 'int'
assign.annotation.ctx = <ast.Load object at 0x100a47f70>
assign.value = <ast.Constant object at 0x103766ef0>
assign.value.value = 1
assign.value.kind = None


## 嵌套类型注解

对于嵌套类型注解, 也就是类似于 ``List[int]``, ``Optional[int]`` 这种, 最关键的就是 value 和 slice 两个属性:

- value 就是括号外面的部分.
- slice 就是括号里面的部分.

In [9]:
# type_def.pyi

import typing as T
from typing import List

a: T.List[int] = [1, 2, 3]
b: List[int] = [4, 5, 6]

In [10]:
source = """
# type_def.pyi

import typing as T
from typing import List

a: T.List[int] = [1, 2, 3]
b: List[int] = [4, 5, 6]
""".strip()

module = ast.parse(source)

# 这里要区分 T.List[int] 和 List[int], 其中 value 的部分 T.List 是一个 ast.Attribute, 而 List 是一个 ast.Name
assign_a = module.body[2]
print(f"--- {assign_a.target.id = }")
anno: ast.Subscript = assign_a.annotation
print(f"{anno = }")
print(f"{anno.value = }")
print(f"{anno.value.value = }")
print(f"{anno.value.attr = }")
print(f"{anno.slice = }")
print(f"{anno.slice.id = }")
print(f"{ast.get_source_segment(source, anno.value)}")

assign_b = module.body[3]
print(f"--- {assign_b.target.id = }")
anno: ast.Subscript = assign_b.annotation
print(f"{anno = }")
print(f"{anno.value = }")
print(f"{anno.value.id = }")
print(f"{anno.slice = }")
print(f"{anno.slice.id = }")
print(f"{ast.get_source_segment(source, anno.value)}")

--- assign_a.target.id = 'a'
anno = <ast.Subscript object at 0x1037ea440>
anno.value = <ast.Attribute object at 0x1037ea620>
anno.value.value = <ast.Name object at 0x1037ea680>
anno.value.attr = 'List'
anno.slice = <ast.Name object at 0x1037ea500>
anno.slice.id = 'int'
T.List
--- assign_b.target.id = 'b'
anno = <ast.Subscript object at 0x1037ea650>
anno.value = <ast.Name object at 0x1037ea710>
anno.value.id = 'List'
anno.slice = <ast.Name object at 0x1037ea890>
anno.slice.id = 'int'
List
